In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2
import nltk
import string
import re
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.base import TransformerMixin
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from tqdm import tqdm_notebook

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [ ]:
train_df = pd.read_csv("train.csv")
train_df.head(5)

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head(5)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [ ]:
y_train = train_df["target"].astype(int)

In [ ]:
train_df.url = train_df.url.str.lower()
test_df.url = test_df.url.str.lower()

In [ ]:
test_df['url'] = test_df['url'].apply(lambda url: "".join(re.sub(r'[^\w\s]+|[\d]+', r'', url).strip()))
train_df['url'] = train_df['url'].apply(lambda url: "".join(re.sub(r'[^\w\s]+|[\d]+', r'', url).strip()))
train_df['url'].head(10)

0                  mkpmd
1                wwwkpby
2           fanserialstv
3          colorboxspbru
4            tulasportru
5         beregifigururu
6    ekbvseinstrumentiru
7           mirtabakaorg
8               xlecxcom
9        busbiletyplusua
Name: url, dtype: object

In [ ]:
train_df.title = train_df.title.str.lower()
test_df.title = test_df.title.str.lower()

In [ ]:
test_df['title'] = test_df['title'].apply(lambda title: "".join(re.sub(r'[^\w\s]+|[\d]+', r'', title).strip()))
train_df['title'] = train_df['title'].apply(lambda title: "".join(re.sub(r'[^\w\s]+|[\d]+', r'', title).strip()))
train_df['title'].head(10)

0    эксминистр экономики молдовы  главе мидэи цель...
1    эта песня стала известна многим телезрителям б...
2    банши  сезон  серия бремя красоты смотреть онлайн
3                                не беси меня картинки
4    в новомосковске сыграют следжхоккеисты алексин...
5    салат корейская морковь  копченая курица  куку...
6    угловой пневмогайковерт fubag rwc    в екатери...
7    табачный магазин мир табака  курительные трубк...
8    league of legends  page   porn comics free online
9         р одесса  ростовнадону  сентября  фаворит чп
Name: title, dtype: object

In [ ]:
train_df['title'] = train_df['title'].apply(word_tokenize)
train_df['title'].head(3)

0    [эксминистр, экономики, молдовы, главе, мидэи,...
1    [эта, песня, стала, известна, многим, телезрит...
2    [банши, сезон, серия, бремя, красоты, смотреть...
Name: title, dtype: object

In [ ]:
test_df["title"] = test_df["title"].apply(word_tokenize)
test_df['title'].head(3)

0    [шестой, кассационный, суд, в, самаре, начнет,...
1    [что, такое, индексация, алиментов, кем, и, в,...
2                      [женщинам, империя, меха, part]
Name: title, dtype: object

In [ ]:
%%time

mystopwords = stopwords.words() + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также', 'т', 'д', 'это', 'который', 'весь', 'свой', 'ещё', 'её', 'ваш', 'такое', 'итак', 'что', 'этот']

train_df['title'] = train_df['title'].apply(lambda title: [word for word in title if word not in mystopwords])
test_df['title'] = test_df['title'].apply(lambda title: [word for word in title if word not in mystopwords])

Wall time: 3min 20s


In [ ]:
%%time

lemmatizer = pymorphy2.MorphAnalyzer()

lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

for i in range(train_df["title"].shape[0]):
    train_df["title"][i] = [lemmatize(word) for word in train_df["title"][i]]
        
for i in range(test_df["title"].shape[0]):
    test_df["title"][i] = [lemmatize(word) for word in test_df["title"][i]]

<timed exec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 9min 2s


In [ ]:
train_df['title'].head(5)

0    [эксминистр, экономика, молдова, глава, мидэи,...
1    [этот, песня, стать, известный, многий, телезр...
2    [банши, сезон, серия, бремя, красота, смотреть...
3                                   [бесить, картинка]
4    [новомосковск, сыграть, следжхоккеисты, алекси...
Name: title, dtype: object

In [ ]:
test_df['title'].head(5)

0    [шестой, кассационный, суд, самара, начать, ра...
1    [индексация, алименты, кто, какой, случай, про...
2                        [женщина, империя, мех, part]
3    [небритый, волосатый, киска, порно, страна, на...
4                                                   []
Name: title, dtype: object

In [ ]:
%%time

for i in range(train_df["title"].shape[0]):
    
    a = []
    b = ''
    for j in train_df["title"][i]:
        a.append(j)
    b = ' '.join(a)
    train_df["title"][i] = b
    
print(train_df["title"])

<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0         эксминистр экономика молдова глава мидэи цель ...
1         этот песня стать известный многий телезритель ...
2           банши сезон серия бремя красота смотреть онлайн
3                                           бесить картинка
4         новомосковск сыграть следжхоккеисты алексински...
                                ...                        
135304    пора тюльпан турецкий сериал русский язык резу...
135305        остросюжетный сериал шеф игра повышение серия
135306    plymouth special deluxe hot rod automaticsmall...
135307    купить skin сыворотка питательный power formul...
135308      технический спирт канистра флакон купить москва
Name: title, Length: 135309, dtype: object
Wall time: 3min 38s


In [ ]:
%%time

for i in range(test_df["title"].shape[0]):
    
    a = []
    b = ''
    for j in test_df["title"][i]:
        a.append(j)
    b = ' '.join(a)
    test_df["title"][i] = b
    
print(test_df["title"])

<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0         шестой кассационный суд самара начать работа р...
1         индексация алименты кто какой случай производи...
2                                  женщина империя мех part
3         небритый волосатый киска порно страна национал...
4                                                          
                                ...                        
165373    armtek запчасть грузовой легковой автомобиль о...
165374    лилия якупова караганда карагандинский область...
165375    администрация лесной район тверской область го...
165376    сонник изменение сознание что сниться изменени...
165377    строительство база команда грандрапидс гриффин...
Name: title, Length: 165378, dtype: object
Wall time: 6min 33s


In [ ]:
class ColumnExtractor(TransformerMixin):
    
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return X[self.column_name].values

In [ ]:
%%time

cv = StratifiedKFold(shuffle = True, n_splits = 3, random_state = 123)

pipline = Pipeline([
        ('features', FeatureUnion([
            ('url', Pipeline([
                ('extractor', ColumnExtractor('url')),
                ('vectorizer', TfidfVectorizer(analyzer = 'char'))])),
            ('title', Pipeline([
                ('extractor', ColumnExtractor('title')),
                ('vectorizer', TfidfVectorizer(analyzer = 'char'))]))])),
        ('clf', LogisticRegression(class_weight = 'balanced', max_iter = 300, random_state = 123))])

params = {'features__url__vectorizer__max_df' : [0.8, 0.9],
          'features__url__vectorizer__min_df' : [4, 5],
          'features__url__vectorizer__ngram_range' : [(3, 4), (4, 4)],
          'features__title__vectorizer__max_df' : [0.8, 0.9],
          'features__title__vectorizer__min_df' : [4, 5],
          'features__title__vectorizer__ngram_range' : [(3, 4), (4, 4)]}

model = GridSearchCV(pipline, params, scoring = 'f1', cv = cv)
model.fit(train_df, y_train)

Wall time: 46min 48s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=123, shuffle=True),
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('url',
                                                                        Pipeline(steps=[('extractor',
                                                                                         <__main__.ColumnExtractor object at 0x0000012A3B96A7C0>),
                                                                                        ('vectorizer',
                                                                                         TfidfVectorizer(analyzer='char'))])),
                                                                       ('title',
                                                                        Pipeline(steps=[('extractor',
                                                                                         <__main__.ColumnExtractor obj...
        

In [ ]:
print(model.best_estimator_)
print(model.best_score_)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('url',
                                                 Pipeline(steps=[('extractor',
                                                                  <__main__.ColumnExtractor object at 0x0000012A44BE56A0>),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(analyzer='char',
                                                                                  max_df=0.8,
                                                                                  min_df=4,
                                                                                  ngram_range=(3,
                                                                                               4)))])),
                                                ('title',
                                                 Pipeline(steps=[('extractor',
        

In [ ]:
%%time

LR_predict = model.best_estimator_.predict(train_df)
LR_predict_proba_train = pd.DataFrame(model.best_estimator_.predict_proba(train_df))
LR_predict_proba_test = pd.DataFrame(model.best_estimator_.predict_proba(test_df))

Wall time: 39.5 s


In [ ]:
f1_score(y_train, LR_predict)

0.9935649168801763

In [ ]:
%%time

params = {'max_depth': [1, 5, 10, 15],
          'n_estimators': [70, 100, 130],
          'learning_rate': [0.01, 0.02, 0.05]}

gbc_model = GridSearchCV(GradientBoostingClassifier(), params, scoring = 'f1', cv = cv)
gbc_model.fit(LR_predict_proba_train, y_train)

Wall time: 24min 37s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=123, shuffle=True),
             estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.02, 0.05],
                         'max_depth': [1, 5, 10, 15],
                         'n_estimators': [70, 100, 130]},
             scoring='f1')

In [ ]:
print(gbc_model.best_estimator_)
print(gbc_model.best_score_)

GradientBoostingClassifier(learning_rate=0.02, max_depth=1)
0.9946196165268892


In [ ]:
%time

GBC_predict_model = gbc_model.best_estimator_.predict(LR_predict_proba_train)

Wall time: 0 ns


In [ ]:
f1_score(y_train, GBC_predict_model)

0.9947681545038717

In [ ]:
GBC_predict_model_ = gbc_model.best_estimator_.predict(LR_predict_proba_test)

In [ ]:
test_df["target"] = GBC_predict_model_.astype(bool)
test_df[["id", "target"]].to_csv("out.csv", index = False)
!cat out.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
